# Exercises for Session 14: Text as Data

In session 14 you will learn how to preprocess text data and structure it, so we can exploit the information in the texts. 

You will work on a larger exercise where you will use the tools you learn. In the exercise you will use the logistic regression model, which is fitted on the movie review dataset in the slides to predict sentiments on a completely different dataset: 

- The dataset contains tweets about US airlines. You can read more about it here: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment?resource=download
- Each tweet has been classified as positive, negative or neutral, which makes it possible for us to compare the predictions from our logistic regression model with the actual sentiments.

The purpose of this exercise is to learn the pitfalls of fitting a model to one kind of text data, and then use the model to predict sentiments of another kind of text data (*cross-domain evaluation*). I.e., we will investigate how *generalizable* our model is.

- Spoiler: The words in the tweets convey a completely different meaning than the words in the movie reviews, so our movie review logistic regression model is not good at predicting sentiments in tweets.

### Fitting the logistic regression model on the movie review data

Before we can get started, you need to run the code below that fits a logistic regression model on the movie review data.

We will then use that model to predict sentiments in the twitter data.

In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Import data
df = pd.read_csv('movie_data.csv', encoding='utf-8', sep=';')

# Clean reviews
def cleaner(document):
    document = document.lower() #To lower case
    document = re.sub(r'<[^>]*>', ' ', document) #Remove HTML
    document = re.sub(r'[^\w\s]','', document) #Remove non-alphanumeric characters
    return document

df['review'] = df['review'].apply(cleaner)

# Load train and test sets to different dataframes
df_train = df[df.set=="train"]
df_test = df[df.set=="test"]

# Sort the data randomly to mix positive and negative reviews
np.random.seed(0)
df_train = df.reindex(np.random.permutation(df_train.index))
df_test = df.reindex(np.random.permutation(df_test.index))

# Take out X and Y variable
x_train = df_train['review'].values
x_test = df_test['review'].values
y_train = df_train['sentiment'].values
y_test = df_test['sentiment'].values

# Make our bag of words
tfidf = TfidfVectorizer()
x_train_bag = tfidf.fit_transform(x_train)

# Fit the model
lr_reviews = LogisticRegression(random_state=0) #Text classifier
lr_reviews.fit(x_train_bag,y_train)

: 

: 

# Part 1: Cross-domain evaluation of a logistic regression model fitted on movie review data

First load the twitter data, which you can find on github under module 14:

In [ ]:
tweet_df = pd.read_csv("AirlineTweets.csv")

Second, change the sentiment to "positive" if the sentiment is labelled "neutral" to have only two categories. In the same time store the sentiments in a list for later use:

In [ ]:
tweet_sentiments = [0 if i=="negative" else 1 for i in tweet_df.airline_sentiment]

Remember to watch the video below before moving on to the exercise:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('piawPVa2Zjk', width=640, height=360)

> **Ex. 14.1.1:** Preprocess the twitter texts. You should at least do the following:
> - Make all letters lower case
> - Remove mentions; i.e. "@user". You can do this with regex.
> - Can you think of other things to clean? Take a look at some of the tweets to look for other unnecessary stuff.

In [ ]:
print(tweet_df.columns)

In [ ]:
import re

# Load and preprocess Twitter data
def preprocess_tweet(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'@user', '', text)  # Remove mentions
    text = re.sub(r'<[^>]*>', ' ', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    # You can add more cleaning steps if necessary
    return text

# Apply preprocessing to the 'tweet' column
tweet_df['text'] = tweet_df['text'].apply(preprocess_tweet)


Remember to watch the video below before moving on to the next exercise:

In [ ]:
YouTubeVideo('UtyYHIDwN8A', width=640, height=360)

> **Ex. 14.1.2:** Make your bag of words from the tweets using the tf-idf vectorizer ("tfidf") previously fitted on the movie review data. 

> *Hint:* You should use the `transform()` method instead of the `fit_transform()` because you have already fitted the vocabulary of the bag of words on the movie review data.

In [ ]:
# Extract the 'text' column from tweet_df
x_tweets = tweet_df['text'].values

# Transform the tweets using the previously fitted tfidf vectorizer
x_tweets_bag = tfidf.transform(x_tweets)

Remember to watch the video below before moving on to the exercise:

In [ ]:
YouTubeVideo('qzSm3rSx2Iw', width=640, height=360)

> **Ex. 14.1.3:** Use the trained logistic regression model from above ("lr_reviews") to predict the sentiment of the tweets.
> - Report testing accuracy

> *Hint:* Use the "tweet_sentiments" list from above to compare the predicted sentiments and the actual sentiments and compute the testing accuracy

In [ ]:
# Predict sentiment using the trained model
y_pred = lr_reviews.predict(x_tweets_bag)

# Compare predicted sentiments with actual sentiments and calculate accuracy
correct_predictions = sum(1 for pred, actual in zip(y_pred, tweet_sentiments) if pred == actual)
total_tweets = len(tweet_sentiments)
accuracy = correct_predictions / total_tweets

print("Testing Accuracy:", accuracy)


> **Ex. 14.1.4:** How well does the logistic regression model from the review data perform in this other domain?
> - Why do you think it does not perform as well?

#### Your answer here

The logistic regression model trained on movie review data may not perform well in Twitter sentiment analysis due to language differences, distinct sentiment expressions, and context variations between movie reviews and tweets. The model's vocabulary might not align with Twitter-specific terms, and its training might not account for Twitter's noisy data, slang, and short text. Additionally, differences in sentiment distribution and feature relevance can contribute to the performance gap. Transferring models between domains often encounters these challenges, requiring domain-specific adjustments for better results.

> **Ex. 14.1.5:** Now train a new logstic regression model on the twitter data:
> 1. Fit a new bag of words on the twitter texts
> 2. Fit a logistic regression model on the twitter bag of words

In [ ]:
# Fit a new bag of words on the twitter texts
tfidf_tweets = TfidfVectorizer()  # Initialize a new TF-IDF vectorizer
x_tweets_bag = tfidf_tweets.fit_transform(x_tweets)  # Fit and transform Twitter texts

# Fit a logistic regression model on the twitter bag of words
lr_tweets = LogisticRegression(random_state=0)  # Initialize a new logistic regression model
lr_tweets.fit(x_tweets_bag, tweet_sentiments)  # Fit the model on the bag of words and sentiment labels

> **Ex. 14.1.6:** What are the most important features/words (high and low coefficients) in the new model? 
> - Do they differ from the most important features from the old model?
> - What does this mean for the models' ability to generalize to new data?

In [ ]:
# Get the feature names from the TF-IDF vectorizer
feature_names = tfidf_tweets.get_feature_names_out()

# Get the coefficients of the logistic regression model
coefficients = lr_tweets.coef_[0]

# Create a dictionary to associate feature names with their coefficients
feature_coefficients = dict(zip(feature_names, coefficients))

# Sort the feature_coefficients dictionary by coefficient values
sorted_features = sorted(feature_coefficients.items(), key=lambda x: x[1], reverse=True)

# Print the top and bottom features
num_features = 10  # Number of features to display
print("Top {} features:".format(num_features))
for feature, coef in sorted_features[:num_features]:
    print(feature, coef)

print("\nBottom {} features:".format(num_features))
for feature, coef in sorted_features[-num_features:]:
    print(feature, coef)